In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 10.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 23.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm import tqdm

from functools import partial
import scipy as sp

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, HistGradientBoostingClassifier, HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/PS-S3/Ep9/train.csv'
file_key_2 = 'Tabular-Playground-Series/PS-S3/Ep9/test.csv'
file_key_3 = 'Tabular-Playground-Series/PS-S3/Ep9/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/PS-S3/Ep9/ConcreteStrengthData.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
train = train.drop(columns = 'id', axis = 1)
train['is_generated'] = 1

test = pd.read_csv(file_content_stream_2)
test['is_generated'] = 1

original = pd.read_csv(file_content_stream_4)
original['is_generated'] = 0
original.columns = train.columns.tolist()

submission = pd.read_csv(file_content_stream_3)

train = pd.concat([train, original], axis = 0)

# Models

In [11]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent']

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

RF_md = RandomForestRegressor(n_estimators = 400,
                              max_depth = 4,
                              min_samples_split = 2,
                              min_samples_leaf = 2).fit(X, Y)
RF_pred = RF_md.predict(test_baseline)
submission['Strength'] = RF_pred
submission.head(10)

,id,Strength
0,5407,46.334474
1,5408,19.244896
2,5409,34.245083
3,5410,46.333850
4,5411,33.994653
5,5412,37.038962
6,5413,34.777062
7,5414,20.468258
8,5415,49.579359
9,5416,41.064615


In [12]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent'] 

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

GBR_md = GradientBoostingRegressor(n_estimators = 500,
                                   learning_rate = 0.01,
                                   max_depth = 2,
                                   min_samples_split = 2,
                                   min_samples_leaf = 2).fit(X, Y)
GBR_pred = GBR_md.predict(test_baseline)
submission['Strength'] = GBR_pred
submission.head(10)

,id,Strength
0,5407,46.526575
1,5408,19.686220
2,5409,33.509168
3,5410,45.523891
4,5411,30.887281
5,5412,38.979399
6,5413,32.989150
7,5414,22.395073
8,5415,47.462149
9,5416,41.116326


In [13]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent'] 

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

hist_md = HistGradientBoostingRegressor(l2_regularization = 0.01,
                                        early_stopping = False,
                                        learning_rate = 0.01,
                                        max_iter = 500,
                                        max_depth = 2,
                                        max_bins = 255,
                                        min_samples_leaf = 10,
                                        max_leaf_nodes = 10).fit(X, Y)

hist_pred = hist_md.predict(test_baseline)
submission['Strength'] = hist_pred
submission.head(10)

,id,Strength
0,5407,46.527786
1,5408,19.748389
2,5409,33.388916
3,5410,45.591218
4,5411,31.164853
5,5412,39.070397
6,5413,33.186932
7,5414,22.752589
8,5415,47.261553
9,5416,41.107063


In [15]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent']

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

lgb_md = LGBMRegressor(n_estimators = 450,
                       max_depth = 3,
                       learning_rate = 0.01,
                       num_leaves = 20,
                       lambda_l1 = 3,
                       lambda_l2 = 3,
                       bagging_fraction = 0.5,
                       feature_fraction = 0.5).fit(X, Y)

lgb_pred = lgb_md.predict(test_baseline)
submission['Strength'] = lgb_pred
submission.head(10)

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5


,id,Strength
0,5407,47.567644
1,5408,20.048230
2,5409,33.610474
3,5410,46.040674
4,5411,29.294744
5,5412,39.764611
6,5413,32.126788
7,5414,21.774256
8,5415,46.927054
9,5416,39.770452


In [17]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent']

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

XGB_md = XGBRegressor(tree_method = 'hist',
                      colsample_bytree = 0.7, 
                      gamma = 0.8, 
                      learning_rate = 0.01, 
                      max_depth = 2, 
                      min_child_weight = 10, 
                      n_estimators = 1000, 
                      subsample = 0.7).fit(X, Y)

xgb_pred = XGB_md.predict(test_baseline)
submission['Strength'] = xgb_pred
submission.head(10)

,id,Strength
0,5407,47.111866
1,5408,18.983866
2,5409,32.288826
3,5410,46.181988
4,5411,28.311680
5,5412,40.252655
6,5413,32.008442
7,5414,22.423332
8,5415,48.696922
9,5416,39.365578


In [18]:
X = train.drop(columns = ['Strength'], axis = 1)
Y = train['Strength']
X['WaterComponent_to_Cement_ratio'] = X['WaterComponent'] / (X['CementComponent'] + 1e-6)
X['Age_Water'] = X['AgeInDays'] / X['WaterComponent']

test_baseline = test.drop(columns = ['id'], axis = 1)
test_baseline['WaterComponent_to_Cement_ratio'] = test_baseline['WaterComponent'] / (test_baseline['CementComponent'] + 1e-6)
test_baseline['Age_Water'] = test_baseline['AgeInDays'] / test_baseline['WaterComponent']

cat_md = CatBoostRegressor(loss_function = 'RMSE',
                           iterations = 1000,
                           learning_rate = 0.01,
                           depth = 3,
                           random_strength = 0.5,
                           bagging_temperature = 0.7,
                           border_count = 30,
                           l2_leaf_reg = 5,
                           verbose = False).fit(X, Y)

cat_pred = cat_md.predict(test_baseline)
submission['Strength'] = cat_pred
submission.head(10)

,id,Strength
0,5407,46.737216
1,5408,18.862649
2,5409,32.376601
3,5410,47.444568
4,5411,31.581712
5,5412,39.476902
6,5413,32.715173
7,5414,22.188544
8,5415,46.478062
9,5416,39.493585


In [21]:
submission['Strength'] = (cat_pred + xgb_pred + hist_pred + GBR_pred) / 4
submission.head()

,id,Strength
0,5407,46.725861
1,5408,19.320281
2,5409,32.890878
3,5410,46.185416
4,5411,30.486381


In [22]:
submission.to_csv('catboost_xgb_hist_GBR_full_original_submission_1.csv', index = False)